In [52]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 

from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings('ignore')

In [53]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

y = data_train.Transported

In [55]:
features = ['CryoSleep', 'Age', 'VIP', 'RoomService', 'VRDeck']
X = data_train[features]
X_test = data_test[features]

In [ ]:
my_model = RandomForestRegressor(n_estimators=100)